# Assignment No 5

Implement the Continuous Bag of Words (CBOW) Model. Stages can be:
a. Data preparation
b. Generate training data
c. Train model
d. Output

In [ ]:
#a. Data preparation

In [23]:
import numpy as np
import keras.backend as K #imp
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
import tensorflow as tf
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
import gensim

In [21]:
data = open(r"/content/corona.txt")
data

<_io.TextIOWrapper name='/content/corona.txt' mode='r' encoding='UTF-8'>

In [22]:
corona_data = [text for text in data if text.count(' ') >= 2]
corona_data

['The speed of transmission is an important point of difference between the two viruses. Influenza has a shorter median incubation period (the time from infection to appearance of symptoms) and a shorter serial interval (the time between successive cases) than COVID-19 virus. The serial interval for COVID-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days. This means that influenza can spread faster than COVID-19. \n',
 'Further, transmission in the first 3-5 days of illness, or potentially pre-symptomatic transmission –transmission of the virus before the appearance of symptoms – is a major driver of transmission for influenza. In contrast, while we are learning that there are people who can shed COVID-19 virus 24-48 hours prior to symptom onset, at present, this does not appear to be a major driver of transmission. \n',
 'The reproductive number – the number of secondary infections generated from one infected individual – is understood to b

In [24]:
vectorize = Tokenizer()
vectorize.fit_on_texts(corona_data)
corona_data = vectorize.texts_to_sequences(corona_data)

total_vocab = sum(len(s) for s in corona_data)
word_count = len(vectorize.index_word)+1
corona_data

[[1,
  38,
  2,
  8,
  9,
  39,
  40,
  41,
  2,
  42,
  13,
  1,
  43,
  23,
  3,
  44,
  11,
  24,
  45,
  46,
  47,
  1,
  14,
  25,
  48,
  10,
  26,
  2,
  27,
  12,
  11,
  24,
  15,
  16,
  1,
  14,
  13,
  49,
  50,
  17,
  4,
  5,
  6,
  1,
  15,
  16,
  7,
  4,
  5,
  6,
  9,
  51,
  10,
  18,
  19,
  52,
  20,
  28,
  7,
  3,
  6,
  1,
  15,
  16,
  9,
  29,
  20,
  30,
  53,
  31,
  3,
  32,
  54,
  55,
  17,
  4,
  5],
 [56,
  8,
  33,
  1,
  57,
  29,
  19,
  20,
  2,
  58,
  59,
  60,
  61,
  62,
  8,
  63,
  2,
  1,
  6,
  64,
  1,
  26,
  2,
  27,
  21,
  9,
  11,
  34,
  35,
  2,
  8,
  7,
  3,
  33,
  65,
  28,
  66,
  22,
  67,
  31,
  68,
  22,
  69,
  70,
  32,
  71,
  4,
  5,
  6,
  72,
  73,
  74,
  75,
  10,
  76,
  77,
  78,
  79,
  30,
  80,
  81,
  82,
  10,
  18,
  11,
  34,
  35,
  2,
  8],
 [1,
  83,
  36,
  21,
  1,
  36,
  2,
  84,
  85,
  86,
  25,
  87,
  88,
  89,
  21,
  9,
  90,
  10,
  18,
  13,
  37,
  12,
  37,
  19,
  7,
  4,
  5,
  6,
  91,
  

In [25]:
print(total_vocab)
print(word_count)

198
103


In [26]:
window_size = 2

In [ ]:
# b. Generate training data

In [27]:
# Defining utility to generate context word pairs
def cbow_model(data, window_size, total_vocab):
    total_length = window_size*2
    for text in data:
        text_len = len(text)
#         print("zero",text)
        for idx, word in enumerate(text):
#             print("first",idx,word)
            context_word = []
            target   = []
            begin = idx - window_size
            end = idx + window_size + 1
            context_word.append([
                text[i]
                for i in range(begin, end)
                if 0 <= i < text_len
                and i != idx
            ])
            target.append(word)
#             print("second",context_word,target)
            contextual = pad_sequences(
                context_word,
                maxlen=total_length
            )
            final_target = tf.keras.utils.to_categorical(
                target,
                total_vocab
            )
#             print("third",contextual,final_target)
            yield(contextual, final_target)

In [ ]:
# c. train model

In [28]:
# Defining the model architecture
model = Sequential()
model.add(
    Embedding(
        input_dim=total_vocab,
        output_dim=100,
        input_length=window_size*2
    )
)
model.add(
    Lambda(
        lambda x: K.mean(x, axis=1),
        output_shape=(100,)
    )
)
model.add(
    Dense(
        total_vocab,
        activation='softmax'
    )
)

In [29]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 4, 100)            19800     
                                                                 
 lambda_1 (Lambda)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 198)               19998     
                                                                 
Total params: 39798 (155.46 KB)
Trainable params: 39798 (155.46 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam'
)

In [31]:
for i in range(10):
    cost = 0
    for x, y in cbow_model(corona_data, window_size, total_vocab):
        cost += model.train_on_batch(x, y)
    print("Epoch ", i,"\t: ", cost)

Epoch  0 	:  1042.0469326972961
Epoch  1 	:  995.4930830001831
Epoch  2 	:  910.9303479194641
Epoch  3 	:  830.8277773857117
Epoch  4 	:  775.1479425430298
Epoch  5 	:  723.6922801733017
Epoch  6 	:  670.464368224144
Epoch  7 	:  615.6918423175812
Epoch  8 	:  560.7597713470459
Epoch  9 	:  507.2302303314209


In [32]:
dimensions = 100
vect_file = open('./vectors.txt','w')
vect_file.write('{} {}\n'.format(102, dimensions))


8

In [33]:
weights = model.get_weights()[0]
for text, i in vectorize.word_index.items():
    final_vec = ' '.join(map(str, list(weights[i, :])))
    vect_file.write('{} {}\n'.format(text, final_vec))
vect_file.close()

In [ ]:
# d. Output

In [34]:
cbow_output = gensim.models.KeyedVectors.load_word2vec_format(
    'vectors.txt',
    binary=False
)

In [35]:
cbow_output.most_similar(positive=['speed'])


[('–transmission', 0.839831531047821),
 ('number', 0.7408813834190369),
 ('difference', 0.7351846694946289),
 ('before', 0.7112886309623718),
 ('of', 0.6805046796798706),
 ('driver', 0.6788857579231262),
 ('serial', 0.6767731308937073),
 ('symptoms', 0.6194480061531067),
 ('first', 0.6076533794403076),
 ('reproductive', 0.5945690870285034)]